In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urlparse
import os
import openai
from dotenv import load_dotenv
from random import sample
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

print(openai.api_key)

In [51]:
def extract_domain(url):
    parsed_url = urlparse(url)
    domain_parts = parsed_url.netloc.split('.')
    subdomain = domain_parts[-3] if len(domain_parts) >= 3 else ''
    main_domain = '.'.join(domain_parts[-2:])
    full_domain = subdomain + '.' + main_domain if subdomain else main_domain
    return full_domain

assert extract_domain("https://cooking.nytimes.com/recipes/1024292-classic-mentaiko-spaghetti?pgType=homepage") == "cooking.nytimes.com"


In [72]:
def extract_from_web(url):
    """Returns the text of a webpage as a string"""
    r = requests.get(url)
    soup = bs(r.content, 'html.parser')

    # matches cooking.nytimes.com
    domain = extract_domain(url)
    print(domain)
    if domain == "cooking.nytimes.com":
        ingredients = soup.find('div', class_='ingredients_ingredients__qAcSs')
        ingredients = ingredients.find_all('li')
        ingredients = [" ".join([sp.text for sp in ing.find_all('span')]) for ing in ingredients]

        steps = soup.find('div', class_='recipebody_prep-block__cpC8w')
        steps = steps.find_all('li')
        steps = [" ".join([p.text for p in step.find_all('p')]) for step in steps]

        summary = soup.find('div', class_="pantry--body topnote_topnoteParagraphs__8kAAN").text
        return (summary + "The ingredients are " + ','.join(ingredients) + "and the steps are " +  ','.join(steps))
    
    elif domain == "www.seriouseats.com":
        ingredients = soup.find('section', class_='section--ingredients')
        ingredients = ingredients.find_all('li')
        ingredients = [ing.get_text().strip() for ing in ingredients]
        
        steps = soup.find('section', id="section--instructions_1-0")
        steps = steps.find_all('li')
        steps = [step.get_text().strip() for step in steps]


        summary = soup.find('div', id="article__header--project_1-0")
        summary = summary.find_all('p')
        summary = "\n".join([p.get_text().strip() for p in summary])

        return (summary + "The ingredients are " +  ','.join(ingredients)  + "and the steps are " + str(steps))
    

    if soup.article:
        soup = soup.article
    elif soup.main:
        soup = soup.main

    ingredient_tags = soup.find(lambda tag: tag.text.lower() in ('ingredients', 'ingredients', 'software'))

    steps_tag = soup.find(lambda tag:  tag.text.lower() in ('preparation', 'steps', 'method', 'directions', 'instructions'))    # 'Modus operandi'

    if steps_tag:
        # Grab the parent element
        parent_element = steps_tag.parent

        # Do something with the parent element
        
        # Come back to this
        return(parent_element.text)


    # return soup.get_text()

In [70]:
# print(extract_from_web("https://cooking.nytimes.com/recipes/1021961-charleston-red-rice"))
# print(extract_from_web("https://www.seriouseats.com/miyeok-guk-korean-seaweed-and-brisket-soup"))
text = extract_from_web("https://www.bonappetit.com/recipe/smash-burger-alfresco")

www.bonappetit.com


In [73]:
print(extract_from_web("https://cooking.nytimes.com/recipes/1021961-charleston-red-rice"))

cooking.nytimes.com
A Lowcountry favorite that likely descended from West African jollof rice, this classic tomato and rice dish is also sometimes called Carolina red rice – or simply red rice. It often contains sausage, but this version, inspired by how the chef Millie Peartree’s mother, Millie Bell, used to make it, relies on bacon for its depth. (Still, if you’d like to add sausage, just let it sear with the vegetables in Step 3.) To start, the “holy trinity” (bell pepper, onion and celery) is cooked in the bacon fat, infusing the cooking liquid. Then, the rice is baked to ensure it’s perfectly cooked. Serve this alongside baked chicken or ham, green beans or collard greens, for a full, satisfying meal.The ingredients are Nonstick cooking spray,6 bacon slices,1 medium Vidalia or other sweet, Spanish or yellow onion, chopped,2 celery ribs, chopped,1 large green bell pepper, chopped,1 teaspoon kosher salt, plus more to taste,2 garlic cloves, minced,2 cups long-grain or Carolina gold r

In [56]:
print(extract_from_web("https://www.seriouseats.com/miyeok-guk-korean-seaweed-and-brisket-soup"))

(['1 ounce (30g) dried\xa0miyeok\xa0seaweed (also sold under the Japanese name\xa0wakame)', '3 whole medium cloves garlic plus 3 finely minced medium cloves garlic, divided', 'One 1-inch piece fresh ginger (about 1/3 ounce; 10g), peeled', '1/2 of a medium white onion (about 3 ounces; 85g for the half onion)', '12 ounces (350g) beef brisket, washed in cold water', '2 tablespoons (30ml) Joseon ganjang (Korean soup soy sauce; see notes), divided', 'Kosher or\xa0sea salt'], ['In a medium bowl, cover seaweed with at least 3 inches cold water and let stand at room temperature until fully softened and hydrated, about 2 hours.', 'Meanwhile, in a Dutch oven or pot, combine whole garlic cloves, ginger, onion, and brisket with 1 1/2 quarts (1 1/2L) cold water and bring to a boil over high heat. Lower heat to maintain a gentle simmer and cook, covered, until brisket is tender and broth is slightly cloudy, about 2 hours. Using a slotted spoon, remove and discard garlic cloves, ginger, and onion fro

In [49]:
str(text)

'<div class="InstructionsWrapper-hZXqPx RmryN" data-testid="InstructionsWrapper"><h2 class="BaseWrap-sc-gjQpdd BaseText-ewhhUZ InstructionListHed-dYZnS iUEiRd kHLSxy eeiAwJ">Preparation</h2><ol class="InstructionGroupWrapper-bqiIwp ccobUj"><li class="InstructionListWrapper-dcpygI bVOrOO"><h4 class="BaseWrap-sc-gjQpdd BaseText-ewhhUZ InstructionHed-cTDCYA InstructionHedSmall-kvDvLP iwCByY kpzQYu ssMUV">Step 1</h4><p>Prepare a grill for high heat. Place a cast-iron griddle, flat side up, on grate; heat until very hot, at least 20 minutes.</p><h4 class="BaseWrap-sc-gjQpdd BaseText-ewhhUZ InstructionHed-cTDCYA InstructionHedSmall-kvDvLP iwCByY kpzQYu ssMUV">Step 2</h4><p>Mix together pickles, mayonnaise, ketchup, adobo sauce, and ½ tsp. salt in a medium bowl; set special sauce aside.</p><h4 class="BaseWrap-sc-gjQpdd BaseText-ewhhUZ InstructionHed-cTDCYA InstructionHedSmall-kvDvLP iwCByY kpzQYu ssMUV">Step 3</h4><p>Divide beef into eight 3-oz. portions (don’t form into patties). Place 4 por

In [52]:
def convert_random_stuff_to_transcript(random_stuff: str):
    functions = [
        {
            "name": "extract_transcript from string",
            "description": "Given a mungled htmk file, convert this into a readable cooking transcript.",
            "parameters": {
                "type": "object",
                    "string_output": {
                        "": {
                            "type": "string",
                            "description": "What does this look like as a string?"
                        },
            },
            "required": ["string_output"],

        }
    }
    ]
    messages = [
        {"role": "system", "content": "You are a helpful assistant that reads this mungled html file, and converts it into a cooking transcript"},
        {"role": "user", "content": 'Here is the mungled html file: ' + random_stuff}
    ]
    response = openai.ChatCompletion.create(
        # model="gpt-4-0613", 
        model="gpt-3.5-turbo-0613", 
        functions=functions,
        messages=messages,
    )

    args = response.choices[0].message.function_call.arguments
    return args
